Zieht sich die Parameterwerte aller Durchläufe in einen pandas dataframe

In [1]:
import pandas as pd
import glob

path =r'./data' # use your path
allFiles = glob.glob(path + "/*.txt")
frame = pd.DataFrame()

#will be converted to dataframe
list_ = []

#dict of style index in frame : (filename, index in file)
findfile = {}

#index for rows in frame
curr_row = 0
for file_ in allFiles:
    orig_ind = 0
    n = 3
    num_lines = sum(1 for l in open(file_))
    skip_idx = [x for x in range(1, num_lines) if x % n != 0]
    df = pd.read_csv(file_,index_col=None, skiprows=skip_idx, header = None)
    list_.append(df)
    for i in range(curr_row, curr_row+len(df.index)):
        findfile[i] = (file_, orig_ind)
        orig_ind += 1
    curr_row +=len(df.index)
frame = pd.concat(list_, ignore_index=True)


Funktion die für gegebenes NNodes, NEdges, phi den Index im Datenframe (der Parameterwerte) ausgibt

In [2]:
def getIndex(NNodes, NEdges, phi):
    lines = []
    for j in range(frame.shape[0]): #all rows
        getstr = frame[0][j]
        #print([int(s) for s in getstr.split() if s.isdigit()])
        if ([int(s) for s in getstr.split() if s.isdigit()][0] == NNodes):
            getstr = frame[1][j]
            if ([int(s) for s in getstr.split() if s.isdigit()][0] == NEdges):
                getstr = frame[2][j]
                if ([int(s) for s in getstr.split() if s.isdigit()][0] == phi):
                    lines.append(j)
    return lines

Funktion die für gegebenen Index des Dataframes (der Parameterwerte) den Meinungsvektor (OporComp = 0) oder den Komponentenvektor (OporComp = 1) aus der jeweiligen Datei liest

In [3]:
#OporComp = 0 for OpinionVector, 1 for ConnectedComponents
def getVector(frameIndex, OporComp):
    filename = (findfile[frameIndex])[0]
    fileindex = (findfile[frameIndex])[1]
    fileline = (fileindex*3)+1
    OpinionVector = pd.read_csv(filename, skiprows = fileline+OporComp, nrows = 1, header=None, delim_whitespace = True)
    return OpinionVector

Funktion die für gegebenes NNodes, NEdges, phi einen Matplotlib subplot eines Plot der Größe x * y an index z erstellt

In [17]:
from collections import Counter
import matplotlib.pyplot as plt

#ComponentSizes dependent on Phi
def histogram(ithplot, axs, NNodes, NEdges, phi):
    hist = [0] * NNodes
    lines = []
    lines = getIndex(NNodes, NEdges, phi)
    if (len(lines) == 0):
        print("No Dataset found for %d, %d, %d" %(NNodes, NEdges, phi))
        return
    for i in range(len(lines)):
        compvec=[NNodes] #empty array
        compvec = getVector(lines[i],1) #getComponentsizevector
        compvec = compvec.values.tolist()[0] #get list from dataframe
        compdic = Counter(compvec) #get Dict of type (compent number) : size
        comps = list(compdic.values()) #get list of size values
        for i in range(len(comps)):
            hist[comps[i]-1] +=1 #-1 because comps may have size 1 and 3200 not 0 and 3199
    hist[:] = [x / len(lines) for x in hist] #average over number of datasets
    #plt.subplot(x, y, z)
    #plt.loglog(hist, 'o')
    #ptitle = "Histogram N:" + str(NNodes) + " M:" +str(NEdges) +"Phi: " +str(float(phi)/100) + " averaged over " + str(len(lines))
    #plt.title(ptitle)
    #plt.xlabel("Component Size")
    #plt.ylabel("Appearance")
    axs = axs,flatten()
    axs[ithplot].plot(hist, 'o')

                

In [18]:
from math import ceil
#philist = [0, 20, 40, 60, 80, 100]
#plt.figure(figsize=(60, 30))
#for index, phi in enumerate(philist):
#    histogram(200, 400, phi, 2, len(philist)/2, index+1)

#plt.show()


philist = [0, 20, 40, 60, 80, 100]
nplots = len(philist)
nrow = ceil(float(nplots/3))
fig, axs = plt.subplots(nrows=nrow, ncols=3, sharex='col', sharey='row', figsize=(10, 40))
for i, alpha in enumerate(philist):
    histogram(i, axs, 200, 400, phi)
plt.show()

NameError: name 'flatten' is not defined